In [ ]:
import sys
import os
import time
import numpy as np
np.set_printoptions(suppress=True, precision=3)

# local .py files
sys.path.append(os.path.abspath(os.path.join('..', 'basics')))
from SceneUR10 import SceneUR10WithObs
from Calcs import get_default_path, upsize_path
from Set_scene import set_scene

from Pre_rrt_star import pre_rrt_star

In [ ]:
scene, targets = set_scene(SceneUR10WithObs)

In [ ]:
import pandas as pd
import json

def save_nodes(nodes_list: list, filename: str):
    nodes_list_to_csv = []
    nodes_list_to_csv.append([0, *nodes_list[0].joints, -1, 0])

    for i in range(1, len(nodes_list)):
        current_node = nodes_list[i]
        nodes_list_to_csv.append([current_node.id, *current_node.joints, current_node.parent.id, current_node.cost])

    columns = ['id', 'joint_1', 'joint_2', 'joint_3', 'joint_4', 'joint_5', 'joint_6', 'parent_id', 'cost']
    df = pd.DataFrame(nodes_list_to_csv, columns=columns)
    df.to_csv(filename + '.csv', header=True, sep=';', index=False)

def save_list(list_to_save: list, filename: str):
    with open(filename, "w") as fp:
        json.dump(list_to_save, fp)

def open_list(filename: str):
    with open(filename, "r") as fp:
        obstacles = json.load(fp)
    return obstacles

In [ ]:
start = np.array([0, 0, 0, 0, 0, 0])
scene.set_joints()

n_paths_per_target = 5
n_goals_for_path = 100
dir_name = 'pre_rrt_star_results'

current_target = 0

for target_index in range(len(targets)):
    target = targets[target_index]
    scene.set_target(target)
    for path_index in range(n_paths_per_target):
        initial_rrt_star_result = False
        while not initial_rrt_star_result == True:
            scene.clear_obs()
            goals = [scene.get_ik() for _ in range(n_goals_for_path)]
            scene.set_joints()

            begin_rrt_star_time = time.time()
            initial_rrt_star_nodes, initial_rrt_star_result, initial_rrt_star_goal_index = pre_rrt_star(scene, start, goals, scene.delta, scene.delta * 20, 3000, goal_bias_prob=0.3)
            end_rrt_star_time = time.time()
        
        rrt_star_time = end_rrt_star_time - begin_rrt_star_time
        rrt_star_n_nodes = len(initial_rrt_star_nodes)
        stats_for_current_path = [rrt_star_time, rrt_star_n_nodes, scene.c_obs]
        print(f'{target_index + 1}_{path_index + 1}')

        full_tree_name = dir_name + '/' + str(target_index + 1) + '_' + str(path_index + 1) + '_tree'
        full_stats_name = dir_name + '/' + str(target_index + 1) + '_' + str(path_index + 1) + '_stats'

        save_nodes(initial_rrt_star_nodes, full_tree_name)
        save_list(stats_for_current_path, full_stats_name)
